In [11]:
from glob import glob
import numpy as np
import cv2

In [12]:
img_path = glob('../data/crop_mask/*.jpg') + glob('../data/crop_without_mask/*.jpg')

In [13]:
d = { 'crop_mask':0 , 'crop_without_mask' : 1}

In [14]:
data = []
target =[]
img_size = 150

for paths in img_path:
    
    img = cv2.imread(paths)
    
    try:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        img_re = cv2.resize(gray,(img_size,img_size))
        
        data.append(img_re)
        
        category = paths.split('/')[2].split('\\')[0]
        target.append(d[category])
    except Exception as e:
        print('Exception',e)

In [15]:
data = np.array(data)/255.0

In [16]:
target = np.array(target)

In [18]:
data = np.reshape(data,(386,150,150,1))

In [19]:
from tensorflow.keras.utils import to_categorical

In [20]:
target_new = to_categorical(target)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

In [31]:
model = Sequential()

In [32]:
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.35))

model.add(Dense(100,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [33]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split( data, target, test_size=0.3, random_state=66)

In [41]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor ='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(X_train,y_train,epochs=20,callbacks=[checkpoint],validation_split=0.3)

Train on 189 samples, validate on 81 samples
Epoch 1/20
189/189 [==============================] - 112s 595ms/sample - loss: 0.6665 - accuracy: 0.6402 - val_loss: 0.6252 - val_accuracy: 0.6790
Epoch 2/20
189/189 [==============================] - 54s 284ms/sample - loss: 0.6268 - accuracy: 0.6772 - val_loss: 0.6085 - val_accuracy: 0.6790
Epoch 3/20
189/189 [==============================] - 68s 362ms/sample - loss: 0.5726 - accuracy: 0.7143 - val_loss: 0.5769 - val_accuracy: 0.7160
Epoch 4/20
189/189 [==============================] - 43s 228ms/sample - loss: 0.5408 - accuracy: 0.7407 - val_loss: 0.5370 - val_accuracy: 0.7160
Epoch 5/20
189/189 [==============================] - 59s 314ms/sample - loss: 0.4466 - accuracy: 0.7989 - val_loss: 0.4932 - val_accuracy: 0.8272
Epoch 6/20
189/189 [==============================] - 47s 249ms/sample - loss: 0.3396 - accuracy: 0.8730 - val_loss: 0.4342 - val_accuracy: 0.8765
Epoch 7/20
189/189 [==============================] - 58s 309ms/sample -

In [42]:
model.evaluate(X_test,y_test)

116/116 [==============================] - 5s 43ms/sample - loss: 0.4766 - accuracy: 0.8707


[0.4766453309305783, 0.87068963]